In [1]:
%cd ..

/home/nikita/edu/competitions/admet


In [2]:
import itertools
import random
import numpy as np
import pandas as pd
import datamol as dm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import RandomOverSampler

from molfeat.calc import RDKitDescriptors2D, FPCalculator, MordredDescriptors
from molfeat.trans import MoleculeTransformer
from molfeat.trans.concat import FeatConcat
from molfeat.trans.fp import FPVecTransformer

from catboost import CatBoostClassifier

In [4]:
df_train_extended = pd.read_csv("submissions/extention_3.csv")
df_train_extended.drop(["mol"], axis=1, inplace=True)
df_train_extended

,Drug_ID,Drug,Y,property
0,0,O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2,1,1
1,1,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0,1
2,2,[N-]=[N+]=CC(=O)NCC(=O)NN,1,1
3,3,[N-]=[N+]=C1C=NC(=O)NC1=O,1,1
4,4,CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O,1,1
...,...,...,...,...
18227,12786,CCN(C)C(C)=NN=Cc1c2c(O)c3c(O)c(C)c4c(c3c1O)C(=...,0,3
18228,12787,CCN(C)C(C)=NN=Cc1c2c(O)c3c(O)c(C)c4c(c3c1O)C(=...,0,3
18229,12788,CCC(C)(CNC(=O)CCCO)c1cccc(OCC)c1,1,3
18230,12789,CCC(C)(CNC(=O)CCCO)c1cccc(OCC)c1,1,3


In [5]:
properties = df_train_extended.property.unique()
dfs_train = [
    df_train_extended[df_train_extended.property == property]
    for property in properties
]

for df in dfs_train:
    print(df.Y.value_counts())


Y
0    5910
1    5470
Name: count, dtype: int64
Y
1    1962
0     291
Name: count, dtype: int64
Y
1    3348
0    1251
Name: count, dtype: int64


In [6]:
mols = dm.from_df(df_train_extended, smiles_column="Drug")

[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not removing hydrogen atom without neighbors
[21:14:42] WARNING: not r

In [7]:
rdkit = FPVecTransformer("desc2D", n_jobs=8, dtype=np.float32, replace_nan=True)
maccs = FPVecTransformer("maccs", dtype=np.float32)
ecfp4 = FPVecTransformer("ecfp:4", dtype=np.float32)

# Wrap the calculator in a transformer instance
featurizer = FeatConcat([maccs, ecfp4, rdkit], dtype=np.float32)

with dm.without_rdkit_log():
    feats = featurizer(mols)

/home/nikita/edu/competitions/admet/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerator
[21:15:00] DEPRECATION WARNING: please use MorganGenerat

In [8]:
df_train_extended.reset_index(drop=True, inplace=True)
df_train_featurized = pd.concat([df_train_extended, pd.DataFrame(feats)], axis=1)

In [9]:
df_train_featurized

,Drug_ID,Drug,Y,property,0,1,2,3,4,5,...,2373,2374,2375,2376,2377,2378,2379,2380,2381,2382
0,0,O=[N+]([O-])c1c2c(c3ccc4cccc5ccc1c3c45)CCCC2,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1,1,O=c1c2ccccc2c(=O)c2c1ccc1c2[nH]c2c3c(=O)c4cccc...,0,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,[N-]=[N+]=CC(=O)NCC(=O)NN,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0
3,3,[N-]=[N+]=C1C=NC(=O)NC1=O,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,0.0,2.0
4,4,CCCCN(CC(O)C1=CC(=[N+]=[N-])C(=O)C=C1)N=O,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18227,12786,CCN(C)C(C)=NN=Cc1c2c(O)c3c(O)c(C)c4c(c3c1O)C(=...,0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,6.0
18228,12787,CCN(C)C(C)=NN=Cc1c2c(O)c3c(O)c(C)c4c(c3c1O)C(=...,0,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,1.0,0.0,6.0
18229,12788,CCC(C)(CNC(=O)CCCO)c1cccc(OCC)c1,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
18230,12789,CCC(C)(CNC(=O)CCCO)c1cccc(OCC)c1,1,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
dfs_train_featurized = [
    df_train_featurized[df_train_featurized.property == property]
    for property in properties
]

In [11]:
feature_cols = df_train_featurized.iloc[:, 4:].columns

In [12]:
forests = []
scalers = []
for i in range(len(properties)):
    model = RandomForestClassifier(
        criterion="entropy", n_estimators=1600, class_weight="balanced", n_jobs=8
    )
    x_train, y_train = dfs_train_featurized[i][feature_cols], dfs_train_featurized[i].Y
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    model.fit(x_train, y_train)
    forests.append(model)
    scalers.append(scaler)
    y_pred = model.predict_proba(x_train)[:, 1]
    auc = roc_auc_score(y_train, y_pred)
    print(f"Property {properties[i]}: {auc:.4f}")

Property 1: 1.0000
Property 2: 1.0000
Property 3: 0.9999


In [13]:
df_test = pd.read_csv("data/test_data.csv", index_col=0)

mols_test = dm.from_df(df_test, smiles_column="Drug")

with dm.without_rdkit_log():
    feats_test = featurizer(mols_test)

df_test.reset_index(drop=True, inplace=True)
df_test_featurized = pd.concat([df_test, pd.DataFrame(feats_test)], axis=1)

[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
[21:16:48] WARNING: not removing hydrogen atom without neighbors
/home/nikita/edu/competitions/admet/.conda/lib/python3.11/site-packages/molfeat/calc/descriptors.py:46: RuntimeWarning: All-NaN slice encountered
  min_charge, max_charge = np.nanmin(atomic_charges), np.nanmax(atomic_charges)
[21:16:49] DEPRECATION WARNING: please use MorganGenerator


In [14]:
preds = []
for i, (model, scaler) in enumerate(zip(forests, scalers)):
    df_subset_test = df_test_featurized[df_test_featurized.property == properties[i]]
    x_test = scaler.transform(df_subset_test[feature_cols])
    y_pred = model.predict_proba(x_test)[:, 1]
    preds.append(y_pred)
preds = np.concatenate(preds, axis=0)

In [15]:
submission = pd.read_csv("data/sample.csv")
submission["Y"] = preds
submission.to_csv("submissions/ilyas3.csv", index=False)
submission

,id,Y
0,0,0.912500
1,1,0.991250
2,2,0.979375
3,3,0.960625
4,4,0.164075
...,...,...
1216,1216,0.016250
1217,1217,0.901875
1218,1218,0.966250
1219,1219,0.919375
